# Fraud Data Exploratory Data Analysis (EDA)

This notebook performs comprehensive EDA on the fraud dataset to understand:
1. Data structure and schema
2. Data quality and missing values
3. Fraud patterns and distributions
4. Temporal analysis
5. Geographic analysis (if available)
6. Value analysis for H1 2023

Run this notebook to understand the data before fixing backend issues.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sqlite3
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)


## 1. Load Data


In [2]:
# Load fraud data from CSV files
print("=== LOADING FRAUD DATA ===")

# Define paths
data_dir = Path("../dataset")
train_file = data_dir / "archive" / "fraudTrain.csv"
test_file = data_dir / "archive" / "fraudTest.csv"

print(f"Data directory: {data_dir}")
print(f"Train file exists: {train_file.exists()}")
print(f"Test file exists: {test_file.exists()}")

# Load data
if train_file.exists():
    train_df = pd.read_csv(train_file)
    print(f"Train data shape: {train_df.shape}")
else:
    print("Train file not found!")
    train_df = None

if test_file.exists():
    test_df = pd.read_csv(test_file)
    print(f"Test data shape: {test_df.shape}")
else:
    print("Test file not found!")
    test_df = None


=== LOADING FRAUD DATA ===
Data directory: ..\dataset
Train file exists: True
Test file exists: True
Train data shape: (1296675, 23)
Test data shape: (555719, 23)


## 2. Basic Data Information


In [3]:
if train_df is not None:
    print("=== TRAIN DATASET BASIC INFO ===")
    print(f"Shape: {train_df.shape}")
    print(f"Columns: {list(train_df.columns)}")
    print(f"\nData types:")
    print(train_df.dtypes)
    print(f"\nMissing values:")
    print(train_df.isnull().sum())
    print(f"\nFirst few rows:")
    train_df.head()


=== TRAIN DATASET BASIC INFO ===
Shape: (1296675, 23)
Columns: ['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud']

Data types:
Unnamed: 0                 int64
trans_date_trans_time     object
cc_num                     int64
merchant                  object
category                  object
amt                      float64
first                     object
last                      object
gender                    object
street                    object
city                      object
state                     object
zip                        int64
lat                      float64
long                     float64
city_pop                   int64
job                       object
dob                       object
trans_num                 object
unix_time                  int64
merch_lat       

## 3. H1 2023 Value Analysis (Critical for Backend Fix)


In [4]:
if train_df is not None and 'trans_date_trans_time' in train_df.columns and 'amt' in train_df.columns:
    print("=== H1 2023 VALUE ANALYSIS ===")
    
    # Filter for H1 2023
    train_df['trans_date_trans_time'] = pd.to_datetime(train_df['trans_date_trans_time'])
    h1_2023 = train_df[(train_df['trans_date_trans_time'] >= '2023-01-01') & 
                       (train_df['trans_date_trans_time'] < '2023-07-01')]
    
    print(f"H1 2023 data shape: {h1_2023.shape}")
    
    if len(h1_2023) > 0:
        # Fraud value analysis
        fraud_data = h1_2023[h1_2023['is_fraud'] == 1]
        total_fraud_value = fraud_data['amt'].sum()
        total_fraud_count = len(fraud_data)
        
        print(f"Total fraud transactions in H1 2023: {total_fraud_count:,}")
        print(f"Total fraud value in H1 2023: ${total_fraud_value:,.2f}")
        
        # High-value vs low-value analysis (proxy for cross-border)
        high_value_threshold = 100
        high_value_fraud = fraud_data[fraud_data['amt'] > high_value_threshold]
        low_value_fraud = fraud_data[fraud_data['amt'] <= high_value_threshold]
        
        high_value_amount = high_value_fraud['amt'].sum()
        low_value_amount = low_value_fraud['amt'].sum()
        
        print(f"\nHigh-value fraud (>${high_value_threshold}):")
        print(f"  Count: {len(high_value_fraud):,}")
        print(f"  Value: ${high_value_amount:,.2f}")
        print(f"  Share: {(high_value_amount/total_fraud_value)*100:.2f}%")
        
        print(f"\nLow-value fraud (≤${high_value_threshold}):")
        print(f"  Count: {len(low_value_fraud):,}")
        print(f"  Value: ${low_value_amount:,.2f}")
        print(f"  Share: {(low_value_amount/total_fraud_value)*100:.2f}%")
        
        # Create the data structure expected by the backend
        value_analysis_data = pd.DataFrame({
            'transaction_type': ['Cross-border (High-value proxy)', 'Domestic (Low-value proxy)'],
            'fraud_value': [high_value_amount, low_value_amount],
            'fraud_count': [len(high_value_fraud), len(low_value_fraud)],
            'percentage_share': [(high_value_amount/total_fraud_value)*100, (low_value_amount/total_fraud_value)*100],
            'total_fraud_value': [total_fraud_value, total_fraud_value]
        })
        
        print(f"\nValue analysis data structure (for backend):")
        print(value_analysis_data)
        
        # Visualize the results
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        # Pie chart of fraud value distribution
        ax1.pie(value_analysis_data['fraud_value'], 
                labels=value_analysis_data['transaction_type'], 
                autopct='%1.1f%%', 
                startangle=90)
        ax1.set_title('H1 2023 Fraud Value Distribution\n(High-value as Cross-border proxy)')
        
        # Bar chart of fraud counts
        ax2.bar(value_analysis_data['transaction_type'], value_analysis_data['fraud_count'])
        ax2.set_title('H1 2023 Fraud Count by Type')
        ax2.set_ylabel('Fraud Count')
        ax2.tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()
        
    else:
        print("No H1 2023 data found!")
else:
    print("Required columns not found for H1 2023 analysis!")


=== H1 2023 VALUE ANALYSIS ===
H1 2023 data shape: (0, 23)
No H1 2023 data found!


## 4. Database Schema Check


In [5]:
print("=== DATABASE SCHEMA CHECK ===")

db_path = "../fraud_data.db"
if Path(db_path).exists():
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Get table info
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print(f"Tables in database: {[table[0] for table in tables]}")
    
    if 'transactions' in [table[0] for table in tables]:
        # Get column info
        cursor.execute("PRAGMA table_info(transactions);")
        columns = cursor.fetchall()
        print(f"\nTransactions table columns:")
        for col in columns:
            print(f"  {col[1]} ({col[2]})")
        
        # Get sample data
        cursor.execute("SELECT * FROM transactions LIMIT 5;")
        sample_data = cursor.fetchall()
        print(f"\nSample data (first 5 rows):")
        for row in sample_data:
            print(f"  {row}")
        
        # Check data counts
        cursor.execute("SELECT COUNT(*) FROM transactions;")
        total_count = cursor.fetchone()[0]
        print(f"\nTotal transactions in database: {total_count:,}")
        
        cursor.execute("SELECT COUNT(*) FROM transactions WHERE is_fraud = 1;")
        fraud_count = cursor.fetchone()[0]
        print(f"Fraud transactions in database: {fraud_count:,}")
        
        # Check H1 2023 data
        cursor.execute("""
            SELECT COUNT(*) FROM transactions 
            WHERE trans_date_trans_time >= '2023-01-01' 
            AND trans_date_trans_time < '2023-07-01'
        """)
        h1_2023_count = cursor.fetchone()[0]
        print(f"H1 2023 transactions: {h1_2023_count:,}")
        
        cursor.execute("""
            SELECT COUNT(*) FROM transactions 
            WHERE trans_date_trans_time >= '2023-01-01' 
            AND trans_date_trans_time < '2023-07-01'
            AND is_fraud = 1
        """)
        h1_2023_fraud_count = cursor.fetchone()[0]
        print(f"H1 2023 fraud transactions: {h1_2023_fraud_count:,}")
        
        # Test the exact SQL query from the backend
        print(f"\n=== TESTING BACKEND SQL QUERY ===")
        test_sql = """
        WITH h1_2023_fraud AS (
            SELECT 
                CASE 
                    WHEN amt > 100 THEN 'Cross-border (High-value proxy)'
                    ELSE 'Domestic (Low-value proxy)'
                END as transaction_type,
                SUM(amt * is_fraud) as fraud_value,
                COUNT(*) as total_transactions,
                SUM(is_fraud) as fraud_count
            FROM transactions
            WHERE trans_date_trans_time >= '2023-01-01' 
                AND trans_date_trans_time < '2023-07-01'
                AND is_fraud = 1
            GROUP BY 
                CASE 
                    WHEN amt > 100 THEN 'Cross-border (High-value proxy)'
                    ELSE 'Domestic (Low-value proxy)'
                END
        ),
        total_fraud AS (
            SELECT SUM(fraud_value) as total_fraud_value
            FROM h1_2023_fraud
        )
        SELECT 
            h.transaction_type,
            h.fraud_value,
            h.fraud_count,
            ROUND((h.fraud_value * 100.0 / t.total_fraud_value), 2) as percentage_share,
            t.total_fraud_value
        FROM h1_2023_fraud h
        CROSS JOIN total_fraud t
        ORDER BY h.fraud_value DESC
        """
        
        try:
            result_df = pd.read_sql_query(test_sql, conn)
            print(f"SQL query executed successfully!")
            print(f"Result shape: {result_df.shape}")
            print(f"Result data:")
            print(result_df)
        except Exception as e:
            print(f"SQL query failed: {e}")
    
    conn.close()
else:
    print(f"Database file not found: {db_path}")


=== DATABASE SCHEMA CHECK ===
Tables in database: ['transactions']

Transactions table columns:
  Unnamed: 0 (INTEGER)
  trans_date_trans_time (TEXT)
  cc_num (INTEGER)
  merchant (TEXT)
  category (TEXT)
  amt (REAL)
  first (TEXT)
  last (TEXT)
  gender (TEXT)
  street (TEXT)
  city (TEXT)
  state (TEXT)
  zip (INTEGER)
  lat (REAL)
  long (REAL)
  city_pop (INTEGER)
  job (TEXT)
  dob (TEXT)
  trans_num (TEXT)
  unix_time (INTEGER)
  merch_lat (REAL)
  merch_long (REAL)
  is_fraud (INTEGER)

Sample data (first 5 rows):
  (0, '2019-01-01 00:00:18', 2703186189652095, 'fraud_Rippin, Kub and Mann', 'misc_net', 4.97, 'Jennifer', 'Banks', 'F', '561 Perry Cove', 'Moravian Falls', 'NC', 28654, 36.0788, -81.1781, 3495, 'Psychologist, counselling', '1988-03-09', '0b242abb623afc578575680df30655b9', 1325376018, 36.011293, -82.048315, 0)
  (1, '2019-01-01 00:00:44', 630423337322, 'fraud_Heller, Gutmann and Zieme', 'grocery_pos', 107.23, 'Stephanie', 'Gill', 'F', '43039 Riley Greens Suite 393', '